Paso 0 — Reset seguro de la colección (opcional)

In [15]:
# --- Paso 0: reset opcional de la colección para evitar "ruido" previo ---
import chromadb

VDB_PATH = "./vectordb"
COLLECTION_NAME = "langchain"

def reset_collection():
    client = chromadb.PersistentClient(path=VDB_PATH)
    try:
        client.delete_collection(COLLECTION_NAME)
        print(f"🧹 Colección '{COLLECTION_NAME}' eliminada.")
    except Exception as e:
        print(f"(info) No se eliminó colección (quizá no existía): {e}")

# Si quieres limpiar todo antes de reindexar:
# reset_collection()


In [ ]:
# --- Paso 1: imports y utilidades ---
import os, re, pdfplumber
from typing import List, Tuple, Dict, Any
from collections import Counter

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document  # langchain_core.documents.Document en versiones nuevas

from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Asegúrate de tener la API en el entorno (no la hardcodees):
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Patrones de limpieza (encabezados/pies repetidos del PDF ejemplo)
CLEAN_PATTERNS = [
    r"FORMATO REQUISICIÓN.*",
    r"ECOPETROL DESARROLLO DE PROYECTO.*",
    r"Todos los derechos reservados.*",
    r"EDP-F-046.*Versión.*",
    r"MR.*#:.*CAS.*",
    r"Página\s+\d+\s+de\s+\d+",
    r"_{6,}",  # líneas de subrayado largas
]

def clean_text(text: str) -> str:
    for p in CLEAN_PATTERNS:
        text = re.sub(p, "", text, flags=re.IGNORECASE)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

# Palabras clave para asignar prioridad de forma genérica
HD_HINTS      = [r"\bAPI\s*675\b", r"HOJA\s*DE\s*DATOS", r"\bDATA\s*SHEET\b", r"BOMBA[S]?\s+(DOSIFICADORA|DOSIFICADORAS)"]
ALCANCE_HINTS = [r"\bALCANCE\b", r"ALCANCE\s+DEL\s+SUMINISTRO"]
REQ_HINTS     = [r"REQUISITO[S]?", r"ESPECIFICACI(ÓN|ONES)", r"CARACTER(Í|I)STIC(A|AS)"]
NORM_HINTS    = [r"\bNORMAS?\b", r"\bESTÁNDARE?S?\b", r"\bRETIE\b", r"\bAPI\b", r"\bASTM\b", r"\bASME\b", r"\bIEC\b", r"\bIEEE\b"]

def matches_any(text: str, patterns: List[str]) -> bool:
    return any(re.search(p, text, flags=re.IGNORECASE) for p in patterns)

def detect_priority(text: str) -> str:
    # Orden de prioridad: hoja_datos > alcance > requisito > norma > general
    if matches_any(text, HD_HINTS):      return "hoja_datos"
    if matches_any(text, ALCANCE_HINTS): return "alcance"
    if matches_any(text, REQ_HINTS):     return "requisito"
    if matches_any(text, NORM_HINTS):    return "norma"
    return "general"

def table_to_text_and_kv(rows: List[List[str]]) -> Tuple[str, Dict[str, str] | None]:
    """Convierte una tabla (lista de filas) a texto legible y, si parece key-value (2 columnas), devuelve un dict kv."""
    # Limpieza básica de celdas
    clean_rows = []
    for r in rows:
        if not r: 
            continue
        rr = [ (c or "").strip() for c in r ]
        if any(rr):
            clean_rows.append(rr)

    kv = None
    # Si parece tabla de 2 columnas (key | value), intenta construir un dict
    if clean_rows and max(len(r) for r in clean_rows) <= 2 and len(clean_rows) >= 2:
        kv = {}
        for r in clean_rows:
            if len(r) >= 2 and r[0] and r[1]:
                kv[r[0]] = r[1]

    text = "\n".join(" | ".join(r) for r in clean_rows)
    return text, (kv if kv else None)


In [17]:
# --- Paso 2: extracción de contenido del PDF ---
import re, json, pdfplumber
from langchain.schema import Document
from typing import List

HD_TOKENS = [
    "TABLA 1. DATOS DE DISEÑO", "HOJA DE DATOS", "API 675",
    "BOMBAS DE DOSIFICACIÓN", "CAUDAL DE INYECCIÓN", "PSI", "GPH"
]

def looks_like_hoja_datos(text: str) -> bool:
    t = text.upper()
    return any(tok in t for tok in HD_TOKENS)

def kv_from_text_page(text: str) -> dict:
    """Extrae pares clave-valor típicos de una Hoja de Datos por regex"""
    t = " ".join(text.split()).upper()
    kv = {}

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*GPH", t)
    if m: kv["caudal_gph"] = m.group(1)

    m = re.search(r"@\s*(\d+(?:[.,]\d+)?)\s*PSI", t)
    if m: kv["presion_psig"] = m.group(1)

    m = re.search(r"(\d+)\s*-\s*(\d+)\s*CPS", t)
    if m:
        kv["rango_cps_min"] = m.group(1)
        kv["rango_cps_max"] = m.group(2)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*HP", t)
    if m: kv["hp_motor"] = m.group(1)

    m = re.search(r"\b(\d+)\s*\+\s*(\d+)\b", t)
    if m:
        kv["bombas_operativas"] = m.group(1)
        kv["bombas_respaldo"] = m.group(2)

    if "304" in t: kv["material_tanque"] = "304SS"
    if "316" in t: kv["material_mojado"] = "316SS"
    if "API 675" in t: kv["api_675"] = "true"

    return kv

def flatten_table_to_text(table_rows):
    lines = []
    for row in table_rows:
        cells = [c.strip() for c in row if c]
        if any(cells):
            lines.append(" | ".join(cells))
    return "\n".join(lines)

def kv_from_table_cells(table_rows):
    return kv_from_text_page(flatten_table_to_text(table_rows))

def extract_pdf_content(filepath: str) -> List[Document]:
    docs: List[Document] = []
    table_settings_list = [
        dict(vertical_strategy="lines", horizontal_strategy="lines",
             intersection_x_tolerance=5, intersection_y_tolerance=5),
        dict(vertical_strategy="text", horizontal_strategy="text")
    ]

    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            txt = page.extract_text() or ""
            clean = clean_text(txt)

            if clean:
                md = {"source": filepath, "page": i, "type": "text"}
                if looks_like_hoja_datos(clean):
                    md["prio"] = "hoja_datos"
                    kv = kv_from_text_page(clean)
                    if kv:
                        md["kv"] = json.dumps(kv, ensure_ascii=False)
                docs.append(Document(page_content=clean, metadata=md))

            for ts in table_settings_list:
                try:
                    tables = page.extract_tables(table_settings=ts) or []
                except Exception:
                    tables = []
                for trows in tables:
                    table_text = flatten_table_to_text(trows)
                    if not table_text.strip():
                        continue
                    md = {"source": filepath, "page": i, "type": "table"}
                    if looks_like_hoja_datos(clean + "\n" + table_text):
                        md["prio"] = "hoja_datos"
                        kv = kv_from_table_cells(trows)
                        if kv:
                            md["kv"] = json.dumps(kv, ensure_ascii=False)
                    docs.append(Document(page_content=table_text, metadata=md))

    return docs




In [18]:
# ============================================
# Paso 3 - Extractor de PDF (texto + tablas + HD)
# ============================================
import pdfplumber
import json
from langchain.schema import Document

def extract_pdf_content(filepath: str):
    docs = []
    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            text = clean_text(text)
            if text:
                docs.append(
                    Document(
                        page_content=text,
                        metadata={"source": filepath, "page": i, "type": "text"}
                    )
                )

            # Tablas
            try:
                tables = page.extract_tables()
                for t in tables:
                    if not t:
                        continue
                    table_text = "\n".join(
                        [" | ".join(filter(None, row)) for row in t if any(row)]
                    )
                    table_text = clean_text(table_text)

                    md = {"source": filepath, "page": i, "type": "table"}

                    # Heurística: detectar Hoja de Datos (API 675, caudal, presión…)
                    kv = {}
                    for row in t:
                        if len(row) >= 2:
                            k, v = row[0], row[1]
                            if k and v:
                                kv[k.strip()] = v.strip()

                    if kv:
                        # 🔑 FIX: serializamos como JSON string
                        md["kv"] = json.dumps(kv, ensure_ascii=False)

                    if table_text:
                        docs.append(Document(page_content=table_text, metadata=md))
            except Exception as e:
                print(f"⚠️ No se pudo procesar tabla en página {i}: {e}")
    return docs


In [23]:
# --- Paso 4: indexar en Chroma (persistente) ---
from collections import Counter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


# --- Paso ******************** función de splitting en chunks ---
def chunk_documents(docs, chunk_size: int = 800, chunk_overlap: int = 100):
    """
    Divide documentos grandes en fragmentos más pequeños para embeddings.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_documents(docs)





def add_files_to_vectordb(filepath: str, reset: bool = False):
    if reset:
        reset_collection()

    raw_docs = extract_pdf_content(filepath)
    splits = chunk_documents(raw_docs)

    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME
    )

    by_type = Counter(d.metadata.get("type", "text") for d in splits)
    by_prio = Counter(d.metadata.get("prio", "general") for d in splits)

    print(f"✅ {len(splits)} fragmentos indexados en {VDB_PATH} (colección {COLLECTION_NAME})")
    print("   Por tipo:", dict(by_type))
    print("   Por prioridad:", dict(by_prio))
    return vectorstore


In [20]:
# --- Paso 5: util para inspeccionar fuentes en la colección ---
def get_unique_sources_list(chroma_settings: chromadb.PersistentClient):
    collection_data = chroma_settings.get_collection(COLLECTION_NAME).get(
        include=['embeddings', 'documents', 'metadatas']
    )
    metadatas = collection_data['metadatas']
    sources = set()
    for m in metadatas:
        if not m: 
            continue
        source = m.get('source', None)
        if source:
            sources.add(source)
    return list(sorted({s.split('/')[-1] for s in sources}))

# Ejemplo:
# client = chromadb.PersistentClient(path=VDB_PATH)
# print("📁 Documentos en vectordb:", get_unique_sources_list(client))


In [24]:
# --- Paso 6: correr el pipeline ---
if __name__ == "__main__":
    # Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf",
        reset=False  # pon True si quieres borrar la colección antes
    )


✅ 613 fragmentos indexados en ./vectordb (colección langchain)
   Por tipo: {'text': 386, 'table': 227}
   Por prioridad: {'general': 613}


In [25]:
# ============================================
# Paso 7 - Debug: explorar fragmentos
# ============================================
import json

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
results = retriever.get_relevant_documents("¿Cuál es el alcance del suministro?")

for r in results:
    print("\n--- CHUNK ---")
    print(r.page_content[:500])
    print("Metadatos:", r.metadata)
    if "kv" in r.metadata:
        try:
            print("KV dict:", json.loads(r.metadata["kv"]))
        except Exception:
            print("KV (raw):", r.metadata["kv"])


/tmp/ipykernel_48323/4084422445.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents("¿Cuál es el alcance del suministro?")



--- CHUNK ---
2. | ALCANCE DEL SUMINISTRO .............................................................................................................. | ............................6
2.1. | ALCANCE GENERAL ................................................................................................................... | ............................6
Metadatos: {'type': 'table', 'page': 2, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK ---
1.3. ORDEN DE PREVALENCIA .....................................................................................................................................5
2. ALCANCE DEL SUMINISTRO ..........................................................................................................................................6
2.1. ALCANCE GENERAL ...............................................................................................................................................6
2.2. EX
Metadato

In [ ]:
# ==========================
# Complemento Paso 7 Función auxiliar de debug
# ==========================
def debug_query(vectorstore, query, prio=None, k=5):
    """Ejecuta una búsqueda y muestra resultados con metadatos."""
    print(f"\n🔎 DEBUG QUERY: '{query}' (prio={prio})\n")
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(query)

    for i, r in enumerate(docs, 1):
        print(f"--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        # Si el chunk viene con metadatos tipo 'kv' (dict convertido a string), lo mostramos aparte
        if "kv" in r.metadata:
            try:
                kv_dict = eval(r.metadata["kv"])
                print("KV dict:", kv_dict)
            except Exception:
                print("KV (raw):", r.metadata["kv"])
        print()




In [ ]:
# Ejemplos:
debug_query(vectorstore, "alcance del suministro", prio="alcance")


🔎 DEBUG QUERY: 'alcance del suministro' (prio=alcance)

--- CHUNK 1 ---
cimentacion | es para todos los equip | os y componen | tes alcance de su | suministro (incluye | soportes).
Metadatos: {'page': 73, 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 2 ---
2. | ALCANCE DEL SUMINISTRO .............................................................................................................. | ............................6 2.1. | ALCANCE GENERAL ................................................................................................................... | ............................6
Metadatos: {'page': 2, 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 3 ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'ty

In [ ]:
debug_query(vectorstore, "API 675 caudal presión material", prio="hoja_datos")


🔎 DEBUG QUERY: 'API 675 caudal presión material' (prio=hoja_datos)

--- CHUNK 1 ---
tolerancia de acuerdo API 675? | SI 103 | Dentro de su oferta se incluye el suministro de las fuerzas y momentos admisibles para todas las conexiones? | SI 104 | Los sellos mecánicos se ofertaron tipo cartucho? | NO 105 | Los acoples de las bombas ofertadas cumplen con el apartado 7.4 de la ET - documento CAS-09991-MER-ET-000002? 106 | El acople cumple con API 671? | NO 107 | El sistema de lubrica
Metadatos: {'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'hoja_datos', 'kv': '{"api_675": "true"}', 'page': 38}
KV dict: {'api_675': 'true'}

--- CHUNK 2 ---
tolerancia de acuerdo API 675? | SI 103 | Dentro de su oferta se incluye el suministro de las fuerzas y momentos admisibles para todas las conexiones? | SI 104 | Los sellos mecánicos se ofertaron tipo cartucho? | NO 105 | Los acoples de las bombas ofertadas cumplen con el apartado 7.4 de la ET - 

In [ ]:
debug_query(vectorstore, "RETIE y normas aplicables", prio="norma")


🔎 DEBUG QUERY: 'RETIE y normas aplicables' (prio=norma)

--- CHUNK 1 ---
regulación vigente.
Metadatos: {'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 6}

--- CHUNK 2 ---
regulación vigente.
Metadatos: {'page': 7, 'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 3 ---
regulación vigente.
Metadatos: {'type': 'text', 'page': 34, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK 4 ---
167 | Todos los gabinetes de control y sus componentes deben demostrar el cumplimiento con los requisitos establecidos en el RETIE mediante un certificado de conformidad de producto expedido por un organismo de certificación debidamente acreditado por la ONAC. | SI | - Servicios 168 | Acompañamiento pre-comisionamiento, comisionamiento y puesta en marcha en el conjunto: Bomba de inyección / Gabine
Metadatos: {'source': './data/CAS09991MERMR000003_Inyeccion

In [ ]:
debug_query(vectorstore, "requisitos técnicos del sistema de dosificación", prio="requisito")


🔎 DEBUG QUERY: 'requisitos técnicos del sistema de dosificación' (prio=requisito)

--- CHUNK 1 ---
requeridos por el cliente para entregar un sistema de dosificación efectivo y que cumpla con las expectativas en la operación y producción de la estación. EDP-F-046 – 20/11/2017 V-1 8/36 Documento Rector: EDP-G-009
Metadatos: {'page': 8, 'prio': 'hoja_datos', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'text'}

--- CHUNK 2 ---
requeridos por el cliente para entregar un sistema de dosificación efectivo y que cumpla con las expectativas en la operación y producción de la estación. EDP-F-046 – 20/11/2017 V-1 8/36 Documento Rector: EDP-G-009
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'text', 'page': 8}

--- CHUNK 3 ---
de acuerdo con la hoja de datos Paquete De Química STAP EC3 CAS-09991-MER-HD-000003 a fin de garantizar una unidad funcional completa. El PROVEEDOR debe cumplir con los requerimi

In [ ]:
# ============================================
# Paso 8 - Probar extracción de Hoja de Datos API 675
# ============================================
import json

def buscar_hojas_datos(pregunta: str = "hoja de datos bomba", k: int = 12):
    # 1) sesgo fuerte a HD via consulta y luego filtramos por metadata
    retriever = vectorstore.as_retriever(search_kwargs={"k": k}, search_type="mmr")
    results = retriever.get_relevant_documents(pregunta)

    # Filtra por prio=hoja_datos o por presencia de "kv"
    filtrados = [r for r in results if (r.metadata.get("prio") == "hoja_datos") or ("kv" in r.metadata)]
    print(f"🔎 Total de chunks candidatos (HD o con KV): {len(filtrados)}")

    count = 0
    for r in filtrados:
        has_kv = "kv" in r.metadata
        if has_kv:
            count += 1
        print("\n--- CHUNK HD ---")
        print(f"p.{r.metadata.get('page')} | {r.metadata.get('type')} | prio={r.metadata.get('prio')}")
        print(r.page_content[:400])

        if has_kv:
            try:
                kv = json.loads(r.metadata["kv"])
                print("KV:", kv)
            except Exception:
                print("KV (raw):", r.metadata["kv"])
    print(f"✅ Chunks con KV reales: {count}")
    return filtrados

# Ejemplo:
buscar_hojas_datos("Datos de diseño de bomba dosificadora API 675 (caudal, presión, HP)")



🔎 Total de chunks candidatos (HD o con KV): 8

--- CHUNK HD ---
p.38 | table | prio=hoja_datos
de diseño y fabricación establecidos por el instituto hidráulico API
675? | SI
95 | El diseño de todos los sistemas de bombeo ofertados permite el
desacople de equipos sin requerir la parada del proceso de
inyección? | NO
96 | La región preferida de las bombas ofertadas se encuentra dentro de
los límites establecidos en el estándar API 675? | SI
97 | Se incluye en la oferta de las bombas la prueba
KV: {'api_675': 'true'}

--- CHUNK HD ---
p.38 | table | prio=hoja_datos
94 | de diseño y fabricación establecidos por el instituto | hidráulico API | SI
675?
El diseño de todos los sistemas de bombeo ofertad | os permite el
95 | desacople de equipos sin requerir la parada del pro | ceso de | NO
96
97
98 | inyección?
La región preferida de las bombas ofertadas se enc
los límites establecidos en el estándar API 675?
Se incluye en la oferta de las bombas la prueba de
UNI
KV: {'api_675': 'true'}

--- C

[Document(metadata={'prio': 'hoja_datos', 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'kv': '{"api_675": "true"}', 'page': 38}, page_content='de diseño y fabricación establecidos por el instituto hidráulico API\n675? | SI\n95 | El diseño de todos los sistemas de bombeo ofertados permite el\ndesacople de equipos sin requerir la parada del proceso de\ninyección? | NO\n96 | La región preferida de las bombas ofertadas se encuentra dentro de\nlos límites establecidos en el estándar API 675? | SI\n97 | Se incluye en la oferta de las bombas la prueba de “COMPLETE\nUNIT TEST”? | NO\n98 | El NPSHR de las bombas ofertadas es al menos 3.3 pies menor al'),
 Document(metadata={'page': 38, 'prio': 'hoja_datos', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'kv': '{"api_675": "true"}', 'type': 'table'}, page_content='94 | de diseño y fabricación establecidos por el instituto | hidráulico API | SI\n675?\nEl diseño de 

In [ ]:
# ============================================
# Paso 9 - Inspeccionar metadatos del Alcance
# ============================================

def buscar_alcance(pregunta: str = "alcance del suministro", k: int = 8):
    """Busca chunks relacionados con alcance y muestra metadatos"""
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Total de chunks recuperados: {len(results)}")
    for r in results:
        print("\n--- CHUNK ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

    return results

# Ejemplo de prueba:
buscar_alcance()


🔎 Total de chunks recuperados: 8

--- CHUNK ---
cimentacion | es para todos los equip | os y componen | tes alcance de su | suministro (incluye | soportes).
Metadatos: {'type': 'table', 'page': 73, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK ---
2. | ALCANCE DEL SUMINISTRO .............................................................................................................. | ............................6 2.1. | ALCANCE GENERAL ................................................................................................................... | ............................6
Metadatos: {'page': 2, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table'}

--- CHUNK ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'type': 'table', 'page': 1, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

---

[Document(metadata={'type': 'table', 'page': 73, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}, page_content='cimentacion | es para todos los equip | os y componen | tes alcance de su | suministro (incluye | soportes).'),
 Document(metadata={'page': 2, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table'}, page_content='2. | ALCANCE DEL SUMINISTRO .............................................................................................................. | ............................6\n2.1. | ALCANCE GENERAL ................................................................................................................... | ............................6'),
 Document(metadata={'type': 'table', 'page': 1, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}, page_content='SECCIÓN | DESCRIPCIÓN\n2 | ALCANCE DEL SUMINISTRO\n10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'),
 

In [ ]:
# ============================================
# Paso 10 - Validar TODOS los metadatos
# ============================================
from collections import Counter

def validar_metadatos_globales():
    # Acceso directo a la colección
    collection = vectorstore._collection
    data = collection.get(include=["metadatas", "documents"])

    total_docs = len(data["documents"])
    print(f"🔎 Total de registros en la colección: {total_docs}")

    errores = 0
    prio_counts = Counter()
    type_counts = Counter()
    kv_count = 0

    for i, meta in enumerate(data["metadatas"]):
        # Validar claves mínimas
        if not all(key in meta for key in ["source", "page", "type"]):
            print(f"⚠️ Registro {i} con metadatos incompletos:", meta)
            errores += 1
            continue

        # Validar valores
        if not isinstance(meta.get("page"), int) or meta["page"] <= 0:
            print(f"⚠️ Registro {i} con página inválida:", meta)
            errores += 1

        if not meta.get("type"):
            print(f"⚠️ Registro {i} con tipo vacío:", meta)
            errores += 1

        # Contadores útiles
        prio_counts[meta.get("priority", "none")] += 1
        type_counts[meta.get("type", "unknown")] += 1
        if "kv" in meta:
            kv_count += 1

    # Resumen
    if errores == 0:
        print("✅ Todos los documentos tienen metadatos completos y válidos.")
    else:
        print(f"⚠️ Se encontraron {errores} problemas de metadatos.")

    print("\n📊 Distribución por prioridad:", dict(prio_counts))
    print("📊 Distribución por tipo:", dict(type_counts))
    print(f"📊 Total de documentos con KV (tablas clave-valor): {kv_count}")

# Ejemplo:
validar_metadatos_globales()


🔎 Total de registros en la colección: 2390
✅ Todos los documentos tienen metadatos completos y válidos.

📊 Distribución por prioridad: {'none': 2390}
📊 Distribución por tipo: {'text': 1046, 'table': 1344}
📊 Total de documentos con KV (tablas clave-valor): 519


In [ ]:
# ============================================
# Paso 11 - Buscar solo tablas o solo texto
# ============================================
def buscar_por_tipo(tipo="table", pregunta="bomba dosificadora"):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5, "filter": {"type": tipo}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados filtrados por tipo='{tipo}': {len(results)}")
    for r in results:
        print("\n--- CHUNK ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

# Ejemplos:
buscar_por_tipo("table", "tags y cantidades")
buscar_por_tipo("text", "alcance del suministro")


🔎 Resultados filtrados por tipo='table': 5

--- CHUNK ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | UND | AX- 75317 | Patín de Preparación y Dosificación: Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss con dos cámaras de preparación, y una cámara para dosificación. Este paquete también incluye un sis
Metadatos: {'kv': '{"material_tanque": "304SS", "api_675": "true"}', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table', 'prio': 'hoja_datos', 'page': 6}

--- CHUNK ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | UND | AX- 75317 | Patín de Preparación y Dosificación: Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss con dos cámaras de preparac

In [ ]:
# ============================================
# Paso 12 - Buscar en una página específica
# ============================================
def buscar_por_pagina(pagina: int, pregunta: str):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5, "filter": {"page": pagina}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados en página {pagina}: {len(results)}")
    for r in results:
        print("\n--- CHUNK ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

# Ejemplo:
buscar_por_pagina(32, "alcance del suministro")
buscar_por_pagina(42, "caudal de la bomba")


🔎 Resultados en página 32: 5

--- CHUNK ---
demás costos ge | nerados y re | queridos para l | levar a ca | bo dicha reparación | y/o | reemplaz | o. 10. ANEXOS 10.1. LISTA D | EL ALCANCE | : El PROVEEDOR | no debe limit | arse a la lista i | ndicada a | continuación, es su | res | ponsabilid | ad entregar un equipo complet | o, probado, | funcional y | listo par | a instalación cump | lien | do con l | as condiciones operacionales in | dic
Metadatos: {'kv': '{"caudal_gph": "18", "hp_motor": "1"}', 'prio': 'hoja_datos', 'page': 32, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table'}

--- CHUNK ---
componentes, o dieciocho (18) meses a partir de la entrega a satisfacción de estos por parte de ECOPETROL S.A. Durante este período la garantía debe cubrir la reparación o reemplazo de los componentes defectuosos por parte del VENDEDOR incluyendo todas las labores, equipos, transporte y demás costos generados y requeridos para llevar a cabo dicha re

In [ ]:
# ============================================
# Paso 13 – Inventario de sistemas (Skids, TAGs y cantidades)
# ============================================

def prueba_inventario_sistemas():
    pregunta = "¿Cuáles son los skids o sistemas de dosificación requeridos? Dame los TAG y las cantidades."
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 10, "filter": {"type": "table"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): {len(results)}")
    for r in results:
        print("\n--- CHUNK INVENTARIO ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)
        if "kv" in r.metadata:
            print("KV:", r.metadata["kv"])

# Ejemplo de ejecución
prueba_inventario_sistemas()


🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): 10

--- CHUNK INVENTARIO ---
SKID. -INSTALACIÓN DE CABLEADOS INTERNOS DE ELÉCTRICIDAD E INSTRUMENTACIÓN, ASÍ COMO INTERCONEXIONES DE LOS EQUIPOS ROTATIVOS Y ESTÁTICOS QUE INTEGRAN EL SKID. -SISTEMA DE CONTROL LOCAL DEL SKID, QUE CUENTEN CON PUERTOS DE FIBRA ÓPTICA Y PROTOCOLO MODBUS TCP/IP PARA COMUNICACIÓN CON EL SISTEMA DE CONTROL DEL STAP EC3; CONTROLADOR PCS-009. 2. DE ACUERDO A INFORMACIÓN DEL PROVEEDOR DEL SKID DE QUÍMI
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 54, 'type': 'table'}

--- CHUNK INVENTARIO ---
95 96 97 98 | 2 | Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss con dos cámaras de preparación, y una camara para dosificación. Este paquete también incluye un sistema de control automático para la entrada de agua, para la introducción del polímero

In [ ]:
# ============================================
# Paso 14 – Alcance + Datos de diseño de la bomba API 675
# ============================================

def prueba_alcance_y_datos_bomba():
    pregunta = "¿Cuál es el alcance del suministro y los datos de diseño de la bomba API 675 (caudal, presión, HP, materiales)?"
    retriever = vectorstore.as_retriever(search_kwargs={"k": 15})
    results = retriever.get_relevant_documents(pregunta)

    alcance = []
    hoja_datos = []

    for r in results:
        if r.metadata.get("type") == "text":
            alcance.append(r)
        if "kv" in r.metadata or r.metadata.get("prio") == "hoja_datos":
            hoja_datos.append(r)

    print(f"🔎 Resultados prueba ALCANCE: {len(alcance)}")
    for r in alcance[:5]:
        print("\n--- CHUNK ALCANCE ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

    print(f"\n🔎 Resultados prueba HOJA DE DATOS: {len(hoja_datos)}")
    for r in hoja_datos[:5]:
        print("\n--- CHUNK HD ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)
        if "kv" in r.metadata:
            print("KV:", r.metadata["kv"])

# Ejemplo de ejecución
prueba_alcance_y_datos_bomba()


🔎 Resultados prueba ALCANCE: 8

--- CHUNK ALCANCE ---
41 Velocidad de giro (RPM) NO - 42 S ane tn ihti od ro a rd ioe ) giro visto desde el extremo del acople (horario/ SI CW 43 Consumo energético (Kw/h) NO - Otros requerimientos Los equipos de bombeo ofertados cumplen con las condiciones de 44 operación y con las condiciones del fluido señaladas en la HD del SI - documentoCAS-09991-MER-HD-000003? Mecánica Características técnicas 45 Código de diseño
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'text', 'page': 38}

--- CHUNK ALCANCE ---
41 Velocidad de giro (RPM) NO - 42 S ane tn ihti od ro a rd ioe ) giro visto desde el extremo del acople (horario/ SI CW 43 Consumo energético (Kw/h) NO - Otros requerimientos Los equipos de bombeo ofertados cumplen con las condiciones de 44 operación y con las condiciones del fluido señaladas en la HD del SI - documentoCAS-09991-MER-HD-000003? Mecánica Características técnicas 45 Código de diseño
Me

In [ ]:
# ============================================
# Paso 14 (v2) – Alcance + Datos de diseño de la bomba API 675 (robusto)
# ============================================
import re, json
from langchain.schema import Document

def _j(x):
    try:
        return json.loads(x)
    except Exception:
        return {}

def _parse_nums(text: str):
    """Extrae caudal (GPH), presión (psig) y HP desde texto libre (fallback)."""
    caudal = None
    pres = None
    hp = None

    # Caudal "12 - 18 GPH" | "12–18 Gph" | "12 a 18 GPH" | también simple "18 GPH"
    m = re.search(r'(\d+(?:[.,]\d+)?)\s*(?:-|–|a|to)\s*(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
    if m:
        caudal = f"{m.group(1)} - {m.group(2)}"
    else:
        m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
        if m:
            caudal = m.group(1)

    # Presión "@ 100 psi" | "100 psig"
    m = re.search(r'@?\s*(\d+(?:[.,]\d+)?)\s*(psi|psig)\b', text, re.I)
    if m:
        pres = m.group(1)

    # Potencia "1 HP"
    m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*hp\b', text, re.I)
    if m:
        hp = m.group(1)

    return caudal, pres, hp

def prueba_alcance_y_datos_bomba_v2():
    q_alc = "alcance del suministro paquete de inyección de químicos"
    q_hd  = "bomba dosificadora API 675 caudal presión HP materiales GPH psig"

    # 1) TWO-RETRIEVERS, con filtros de prioridad donde aplique
    retr_alc = vectorstore.as_retriever(
        search_kwargs={"k": 12, "filter": {"prio": "alcance"}},
        search_type="mmr"
    )
    alc_docs = retr_alc.get_relevant_documents(q_alc)

    # Fallback si no hay prio=alcance (algunos PDFs no quedaron marcados)
    if not alc_docs:
        retr_alc_fb = vectorstore.as_retriever(search_kwargs={"k": 12}, search_type="mmr")
        alc_docs = retr_alc_fb.get_relevant_documents("alcance del suministro")

    retr_hd = vectorstore.as_retriever(
        search_kwargs={"k": 20, "filter": {"prio": "hoja_datos"}},
        search_type="mmr"
    )
    hd_docs = retr_hd.get_relevant_documents(q_hd)

    # 2) Refuerzo por metadatos de colección (trae HD con KV aunque el retriever no lo devuelva)
    coll = vectorstore._collection
    meta_hd = coll.get(where={"prio": "hoja_datos"}, include=["metadatas", "documents"])
    for md, doc in zip(meta_hd["metadatas"], meta_hd["documents"]):
        if not isinstance(md, dict):
            continue
        if ("kv" in md and any(k in md["kv"] for k in ["caudal_gph", "presion_psig", "hp_motor", "material_tanque", "material_mojado"])) \
           or ("API 675" in (doc or "") or "GPH" in (doc or "") or "psig" in (doc or "")):
            hd_docs.append(Document(page_content=doc, metadata=md))

    # 3) De-duplicar por (page,type,primeros 60 chars)
    def _key(d): return (d.metadata.get("page"), d.metadata.get("type"), (d.page_content or "")[:60])
    seen = set()
    alc_unique, hd_unique = [], []
    for d in alc_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); alc_unique.append(d)
    for d in hd_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); hd_unique.append(d)

    # 4) Extraer KV / texto -> síntesis
    datos = {
        "caudal_gph": None,
        "presion_psig": None,
        "hp_motor": None,
        "materiales": set(),
        "api_675": False,
        "paginas_fuente": set(),
    }

    for d in hd_unique:
        md = d.metadata or {}
        if "page" in md:
            datos["paginas_fuente"].add(md["page"])

        # KV directo
        if "kv" in md:
            kv = _j(md["kv"])
            if not datos["caudal_gph"] and kv.get("caudal_gph"):
                datos["caudal_gph"] = kv["caudal_gph"]
            if not datos["presion_psig"] and kv.get("presion_psig"):
                datos["presion_psig"] = kv["presion_psig"]
            if not datos["hp_motor"] and kv.get("hp_motor"):
                datos["hp_motor"] = kv["hp_motor"]
            if kv.get("material_tanque"):
                datos["materiales"].add(kv["material_tanque"])
            if kv.get("material_mojado"):
                datos["materiales"].add(kv["material_mojado"])
            if str(kv.get("api_675")).lower() == "true":
                datos["api_675"] = True

        # Fallback regex en texto
        if any(v is None for v in (datos["caudal_gph"], datos["presion_psig"], datos["hp_motor"])):
            c, p, h = _parse_nums(d.page_content or "")
            if c and not datos["caudal_gph"]:
                datos["caudal_gph"] = c
            if p and not datos["presion_psig"]:
                datos["presion_psig"] = p
            if h and not datos["hp_motor"]:
                datos["hp_motor"] = h

    # 5) Mostrar resultados
    print(f"🔎 ALCANCE (k={len(alc_unique)}):")
    for r in alc_unique[:5]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))

    print(f"\n🔎 HOJA DE DATOS (k={len(hd_unique)}):")
    for r in hd_unique[:6]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))
        if "kv" in r.metadata:
            print("KV:", r.metadata["kv"])

    print("\n✅ EXTRACCIÓN SINTETIZADA:")
    print(f"  • Caudal (GPH): {datos['caudal_gph']}")
    print(f"  • Presión (psig): {datos['presion_psig']}")
    print(f"  • Potencia (HP): {datos['hp_motor']}")
    print(f"  • Materiales: {', '.join(sorted(datos['materiales'])) or '—'}")
    print(f"  • API 675: {'Sí' if datos['api_675'] else 'N/D'}")
    print(f"  • Páginas fuente (HD): {sorted(datos['paginas_fuente'])}")

    return {"alcance": alc_unique, "hoja_datos": hd_unique, "datos": datos}

# Ejecución:
prueba_alcance_y_datos_bomba_v2()


🔎 ALCANCE (k=12):

— p.73 | table
cimentacion | es para todos los equip | os y componen | tes alcance de su | suministro (incluye | soportes).

— p.1 | table
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)

— p.2 | text
1.3. ORDEN DE PREVALENCIA .....................................................................................................................................5 2. ALCANCE DEL SUMINISTRO ..........................................................................................................................................6 2.1. ALCANCE GENERAL ..

— p.18 | table
responsabilidad | por la ca | lidad de cada | componente q | ue lo conforma qu | e haga parte | de | l alcance del suministro, de | acuerdo | con lo indicad | o en la presen | te requisición de m | ateriales y s | us | documentos anexos. Ademá | s, deberá | tener la autoriz | ación de cumpl | imiento de garantía | asignada por | el | FABRICANTE de 



{'alcance': [Document(metadata={'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'page': 73}, page_content='cimentacion | es para todos los equip | os y componen | tes alcance de su | suministro (incluye | soportes).'),
  Document(metadata={'page': 1, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'table'}, page_content='SECCIÓN | DESCRIPCIÓN\n2 | ALCANCE DEL SUMINISTRO\n10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'),
  Document(metadata={'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'type': 'text', 'page': 2}, page_content='1.3. ORDEN DE PREVALENCIA .....................................................................................................................................5\n2. ALCANCE DEL SUMINISTRO ..........................................................................................................................................6\n2.1. 